In [1]:
import sys
import time
import warnings
import logging
import config
import math
import torch
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from functools import reduce
from collections import defaultdict
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModel
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch.optim.optimizer import Optimizer
from torch.optim.optimizer import required
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import (accuracy_score, roc_curve, auc)
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

I0729 10:51:17.035197 14712 file_utils.py:39] PyTorch version 1.2.0 available.
I0729 10:51:28.004090 14712 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at C:\Users\YangWang/.cache\torch\transformers\a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
I0729 10:51:28.008088 14712 configuration_utils.py:321] Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "vocab_size": 30522
}

I0729 10:51:28.873129 14712 token

In [2]:
from deep_portfolio import ReutersClassifier

model = ReutersClassifier(n_classes=2, top_k=10, p=0.1, window_size=3, out_channels=64)
model.load_state_dict(torch.load("../weights/distilbert.bin"))
model.eval()

I0729 10:55:10.747162 14712 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at C:\Users\YangWang/.cache\torch\transformers\a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
I0729 10:55:10.751210 14712 configuration_utils.py:321] Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "vocab_size": 30522
}

I0729 10:55:11.164169 14712 modeling_utils.py:650] loading weights file https://cdn.huggingface.co/distilbert-ba

RuntimeError: Error(s) in loading state_dict for ReutersClassifier:
	Missing key(s) in state_dict: "cnn_max_pool.cnn.weight", "cnn_max_pool.cnn.bias". 
	size mismatch for classifier.weight: copying a param with shape torch.Size([2, 2304]) from checkpoint, the shape in current model is torch.Size([2, 64]).

In [6]:
df = joblib.load("../data/sp500_top100_v1.bin")
df.head()

,title,date,query,url,ticker
0,BRIEF-Apple Inc Says Not Allowing Entertainmen...,2020-03-15,Apple Inc.,https://www.reuters.com/article/idUSFWN2B61K2,AAPL
1,Apple signs multi-year deals with major music ...,2020-03-12,Apple Inc.,https://www.reuters.com/article/idUSKBN20Z33J,AAPL
2,Apple signs multi-year deals with major music ...,2020-03-12,Apple Inc.,https://www.reuters.com/article/idUSL4N2B54T2,AAPL
3,Chinese regulators remove 'Plague Inc' game fr...,2020-02-28,Apple Inc.,https://www.reuters.com/article/idUSKCN20M043,AAPL
4,UPDATE 1-Chinese regulators remove 'Plague Inc...,2020-02-28,Apple Inc.,https://www.reuters.com/article/idUSL3N2AS0OO,AAPL


In [8]:
train = joblib.load("../data/train_top10_v2.bin")
train.head()

,Top 1 News,Top 2 News,Top 3 News,Top 4 News,Top 5 News,Top 6 News,Top 7 News,Top 8 News,Top 9 News,Top 10 News,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,label
date,,,,,,,,,,,,,,,,,,,
2012-01-04,apple hires adobe officer to lead iad: report,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.71,51.28,50.62,51.13,65005500.0,0.0,0.0,AAPL,1
2012-01-05,refile-taiwan is elan says apple to pay $5 mln...,taiwan is elan says apple to pay $5 million in...,update 1-apple to pay elan $5 mln to settle pa...,apple to pay $5 million to settle patent lawsu...,NaN,NaN,NaN,NaN,NaN,NaN,51.32,51.76,51.04,51.70,67817400.0,0.0,0.0,AAPL,1
2012-01-09,apple is cook got rich stock award worth $376 ...,apple is siri puts voice-enabled search in spo...,chinese authors sue apple for copyright infrin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.62,52.90,52.11,52.16,98506100.0,0.0,0.0,AAPL,0
2012-01-10,"kodak sues apple, htc over digital image patents",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.67,52.68,52.13,52.34,64549100.0,0.0,0.0,AAPL,1
2012-01-11,strong apple contingent expected at ces,corrected-ces-strong apple contingent expected...,apple plans january 19 education event in new ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.27,52.29,51.86,52.26,53771200.0,0.0,0.0,AAPL,0
